# ADS / IDS Annotation Tool

**Everything happens here** — listen to segments and label them without leaving this notebook.

### Workflow
1. **Cell 2** — Configure participant, speaker & parquet path
2. **Cell 3** — Load audio & segment boundaries from Stage 03 parquet
3. **Cell 4** — Annotate: plays each segment, you type `0` / `1` / `2` and press Enter
4. **Cell 5** — Export labeled ADS & IDS WAV files

### Labels
| Input | Label |
|-------|-------|
| `0` | Other |
| `1` | ADS (Adult-Directed Speech) |
| `2` | IDS (Infant-Directed Speech) |
| `r` | Replay current segment |
| `b` | Go back one segment |
| `q` | Save & stop (resume later) |

In [ ]:
# ── Configuration ──────────────────────────────────────────────
PARTICIPANT_ID = "ABAN141223"       # ← change this
SPEAKER        = "female"           # "female" or "male"

# Paths (no need to change unless non-default)
CLASSIFIED_ROOT  = "/scratch/users/arunps/hindibabynet/audio_classified"
ANNOTATION_ROOT  = "/scratch/users/arunps/hindibabynet/annotations"
ARTIFACTS_ROOT   = "../artifacts/runs"   # relative to notebook

# Parquet path – set to "" to auto-discover the latest run
PARQUET_PATH = ""  # e.g. "../artifacts/runs/20260212_134452/speaker_classification/ABAN141223_segments.parquet"

# Gap inserted between segments when pipeline built main_female/male.wav
STREAM_GAP_SEC = 0.15

In [ ]:
import csv, wave, time
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import IPython.display as ipd
from IPython.display import display, clear_output

LABEL_MAP = {0: "Other", 1: "ADS", 2: "IDS"}
SPEAKER_CLASS_MAP = {"female": "adult_female", "male": "adult_male"}

# ── Read WAV ──────────────────────────────────────────────────
def read_wav(path):
    with wave.open(str(path), "rb") as wf:
        sr = wf.getframerate()
        raw = wf.readframes(wf.getnframes())
    return np.frombuffer(raw, dtype=np.int16).astype(np.float32) / 32768.0, sr

# ── Parquet-based segment loading ─────────────────────────────
def find_segments_parquet(artifacts_root: str, participant_id: str) -> Optional[Path]:
    """Auto-discover the latest Stage 03 segments parquet for a participant."""
    root = Path(artifacts_root)
    pattern = f"*/speaker_classification/{participant_id}_segments.parquet"
    candidates = sorted(root.glob(pattern))
    return candidates[-1] if candidates else None

def load_segments_from_parquet(
    parquet_path: Path,
    speaker: str,
    gap_sec: float = 0.15,
) -> Tuple[List[Tuple[float, float]], pd.DataFrame]:
    """
    Load classified segments from the Stage 03 parquet and reconstruct
    their positions inside main_female.wav / main_male.wav.

    The pipeline's build_class_stream concatenates segments (sorted by
    start_sec) with gap_sec silence gaps. We replicate that to compute
    each segment's (stream_start, stream_end) inside the WAV.
    """
    target_class = SPEAKER_CLASS_MAP[speaker]
    df = pd.read_parquet(parquet_path)

    seg_df = (
        df[df["predicted_class"] == target_class]
        .copy()
        .sort_values("start_sec")
        .reset_index(drop=True)
    )

    if seg_df.empty:
        return [], seg_df

    segments: List[Tuple[float, float]] = []
    cursor = 0.0
    for _, row in seg_df.iterrows():
        dur = float(row["duration_sec"])
        segments.append((cursor, cursor + dur))
        cursor += dur + gap_sec

    return segments, seg_df

# ── CSV persistence ──────────────────────────────────────────
def csv_path(pid, speaker):
    return Path(ANNOTATION_ROOT) / pid / f"{pid}_{speaker}_annotations.csv"

def load_ann(pid, speaker, segments):
    p = csv_path(pid, speaker)
    if not p.exists(): return {}
    ann = {}
    with open(p, newline="") as f:
        for row in csv.DictReader(f):
            ann[int(row["segment_index"])] = int(row["label"])
    return ann

def save_ann(pid, speaker, segments, annotations, seg_df=None):
    p = csv_path(pid, speaker)
    p.parent.mkdir(parents=True, exist_ok=True)
    has_meta = seg_df is not None and not seg_df.empty
    with open(p, "w", newline="") as f:
        w = csv.writer(f)
        header = ["segment_index", "start_sec", "end_sec", "duration_sec", "label", "label_name"]
        if has_meta:
            header += ["orig_start_sec", "orig_end_sec", "chunk_id", "predicted_confidence"]
        w.writerow(header)
        for idx in sorted(annotations.keys()):
            if idx < len(segments):
                s, e = segments[idx]
                lab = annotations[idx]
                row = [idx, f"{s:.4f}", f"{e:.4f}", f"{e-s:.4f}", lab, LABEL_MAP[lab]]
                if has_meta and idx < len(seg_df):
                    r = seg_df.iloc[idx]
                    row += [
                        f"{r['start_sec']:.4f}",
                        f"{r['end_sec']:.4f}",
                        int(r["chunk_id"]) if "chunk_id" in r.index else "",
                        f"{r['predicted_confidence']:.4f}" if "predicted_confidence" in r.index else "",
                    ]
                w.writerow(row)
    return p

# ── Load audio & segments from parquet ────────────────────────
wav_path = Path(CLASSIFIED_ROOT) / PARTICIPANT_ID / f"{PARTICIPANT_ID}_main_{SPEAKER}.wav"
assert wav_path.exists(), f"File not found: {wav_path}"

print(f"Loading {wav_path.name} ...")
audio, sr = read_wav(wav_path)
dur = len(audio) / sr
print(f"Duration: {dur:.1f}s ({dur/60:.1f} min)  |  SR: {sr} Hz")

# Resolve parquet
if PARQUET_PATH:
    pq_path = Path(PARQUET_PATH)
else:
    pq_path = find_segments_parquet(ARTIFACTS_ROOT, PARTICIPANT_ID)
assert pq_path is not None and pq_path.exists(), (
    f"Segments parquet not found for {PARTICIPANT_ID}.\n"
    f"  Searched: {ARTIFACTS_ROOT}/*/speaker_classification/{PARTICIPANT_ID}_segments.parquet\n"
    f"  Set PARQUET_PATH in the config cell to specify explicitly."
)
print(f"Using parquet: {pq_path}")

print(f"Loading segments from parquet (class={SPEAKER_CLASS_MAP[SPEAKER]}) ...")
segments, seg_df = load_segments_from_parquet(pq_path, SPEAKER, gap_sec=STREAM_GAP_SEC)
durs = [e - s for s, e in segments]
print(f"Found {len(segments)} segments  |  "
      f"min={min(durs):.2f}s  median={sorted(durs)[len(durs)//2]:.2f}s  max={max(durs):.2f}s")

# Load existing annotations
annotations = load_ann(PARTICIPANT_ID, SPEAKER, segments)
if annotations:
    print(f"Loaded {len(annotations)} existing annotations (will resume from where you left off)")
print(f"\n✓ Ready — run the next cell to start annotating.")

In [ ]:
# ── Annotate: listen + label in one loop ─────────────────────
# Re-run this cell to resume from where you left off.

total = len(segments)
# Find first unannotated segment
start_idx = 0
unannotated = [i for i in range(total) if i not in annotations]
if unannotated:
    start_idx = unannotated[0]
else:
    print(f"All {total} segments already annotated! Run the export cell below.")

n_ads = sum(1 for v in annotations.values() if v == 1)
n_ids = sum(1 for v in annotations.values() if v == 2)
n_oth = sum(1 for v in annotations.values() if v == 0)
print(f"Progress: {len(annotations)}/{total} annotated  |  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
print(f"Starting from segment {start_idx + 1}\n")
print("Labels:  0=Other  1=ADS  2=IDS  |  r=replay  b=back  q=save & stop\n")

idx = start_idx
try:
    while 0 <= idx < total:
        s, e = segments[idx]
        dur_s = e - s
        existing = annotations.get(idx)
        tag = f"  [current: {LABEL_MAP[existing]}]" if existing is not None else ""

        # Show original-recording times from parquet if available
        orig_info = ""
        if not seg_df.empty and idx < len(seg_df):
            r = seg_df.iloc[idx]
            orig_info = f"  (orig: {r['start_sec']:.2f}s–{r['end_sec']:.2f}s)"

        # Extract segment audio
        s_samp = max(0, int(s * sr))
        e_samp = min(len(audio), int(e * sr))
        chunk = audio[s_samp:e_samp]

        clear_output(wait=True)
        n_ads = sum(1 for v in annotations.values() if v == 1)
        n_ids = sum(1 for v in annotations.values() if v == 2)
        n_oth = sum(1 for v in annotations.values() if v == 0)
        print(f"[{idx+1}/{total}]  {s:.2f}s – {e:.2f}s  ({dur_s:.2f}s){tag}{orig_info}")
        print(f"Progress: {len(annotations)}/{total}  |  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
        print("─" * 50)

        # Play audio (autoplay in browser)
        display(ipd.Audio(chunk, rate=sr, autoplay=True))

        while True:
            raw = input("Label (0/1/2/r/b/q): ").strip().lower()
            if raw == "q":
                save_ann(PARTICIPANT_ID, SPEAKER, segments, annotations, seg_df)
                clear_output(wait=True)
                n_ads = sum(1 for v in annotations.values() if v == 1)
                n_ids = sum(1 for v in annotations.values() if v == 2)
                n_oth = sum(1 for v in annotations.values() if v == 0)
                print(f"✓ Saved {len(annotations)}/{total} annotations.  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
                print(f"  CSV: {csv_path(PARTICIPANT_ID, SPEAKER)}")
                print(f"  Re-run this cell to resume.")
                raise StopIteration
            elif raw == "r":
                clear_output(wait=True)
                print(f"[{idx+1}/{total}]  {s:.2f}s – {e:.2f}s  ({dur_s:.2f}s)  (replaying...)")
                print(f"Progress: {len(annotations)}/{total}  |  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
                print("─" * 50)
                display(ipd.Audio(chunk, rate=sr, autoplay=True))
                continue
            elif raw == "b":
                if idx > 0:
                    idx -= 1
                break
            elif raw in ("0", "1", "2"):
                annotations[idx] = int(raw)
                idx += 1
                # Auto-save every 10
                if len(annotations) % 10 == 0:
                    save_ann(PARTICIPANT_ID, SPEAKER, segments, annotations, seg_df)
                break
            else:
                print("  Invalid. Use 0, 1, 2, r, b, or q.")

    # Finished all segments
    save_ann(PARTICIPANT_ID, SPEAKER, segments, annotations, seg_df)
    clear_output(wait=True)
    n_ads = sum(1 for v in annotations.values() if v == 1)
    n_ids = sum(1 for v in annotations.values() if v == 2)
    n_oth = sum(1 for v in annotations.values() if v == 0)
    print(f"🎉 All {total} segments annotated!  ADS={n_ads}  IDS={n_ids}  Other={n_oth}")
    print(f"CSV: {csv_path(PARTICIPANT_ID, SPEAKER)}")
    print(f"Run the export cell below to create separate WAV files.")

except StopIteration:
    pass
except KeyboardInterrupt:
    save_ann(PARTICIPANT_ID, SPEAKER, segments, annotations, seg_df)
    print(f"\nInterrupted — saved {len(annotations)} annotations. Re-run to resume.")

In [ ]:
# ── Export: create separate ADS / IDS / Other WAV files ──────
# Run this after annotation is complete (or partially complete).

def write_wav16(path, data, sr):
    path.parent.mkdir(parents=True, exist_ok=True)
    pcm = np.clip(data * 32768.0, -32768, 32767).astype(np.int16)
    with wave.open(str(path), "wb") as wf:
        wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(sr)
        wf.writeframes(pcm.tobytes())

out_dir = Path(ANNOTATION_ROOT) / PARTICIPANT_ID
gap = np.zeros(int(sr * 0.15), dtype=np.float32)

n_ann = len(annotations)
n_ads = sum(1 for v in annotations.values() if v == 1)
n_ids = sum(1 for v in annotations.values() if v == 2)
n_oth = sum(1 for v in annotations.values() if v == 0)
print(f"Annotations: {n_ann}/{len(segments)}  |  ADS={n_ads}  IDS={n_ids}  Other={n_oth}\n")

for label_code, label_name in LABEL_MAP.items():
    pieces = []
    for idx, lab in sorted(annotations.items()):
        if lab != label_code or idx >= len(segments): continue
        s, e = segments[idx]
        chunk = audio[max(0, int(s*sr)):min(len(audio), int(e*sr))]
        if len(chunk) > 0:
            pieces.append(chunk)
            pieces.append(gap)
    if pieces:
        combined = np.concatenate(pieces)
        out_path = out_dir / f"{PARTICIPANT_ID}_{SPEAKER}_{label_name}.wav"
        write_wav16(out_path, combined, sr)
        out_dur = len(combined) / sr
        print(f"  {label_name:6s} → {out_path}  ({out_dur:.1f}s)")
    else:
        print(f"  {label_name:6s} → (no segments)")

print(f"\n✓ Export complete.")